In [8]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.9 MB/s eta 0:00:0000:0100:01


In [34]:
from transformers import pipeline
import os
import getpass
import openai
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import math
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pypdf import PdfReader

[nltk_data] Downloading package punkt to /Users/shruti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
import re
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

def extract_chapters_from_index(pdf_path):
    index_headings = re.compile(r'(Contents|Chapters|Index)', re.IGNORECASE)
    chapter_pattern = re.compile(r'Chapter \d+', re.IGNORECASE)
    chapter_titles = []
    capture_titles = False
    found_index_page = False

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                text = element.get_text()
                if index_headings.search(text):
                    capture_titles = True
                    found_index_page = True
                    continue
                if capture_titles:
                    for line in text.split('\n'):
                        if line.strip() and not line.strip().isdigit():
                            chapter_titles.append(line.strip())
        
        if capture_titles:
            break

    # Fallback: If no index page is found, look for chapters throughout the PDF
    if not found_index_page:
        for page_layout in extract_pages(pdf_path):
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    text = element.get_text()
                    found_chapters = chapter_pattern.findall(text)
                    chapter_titles.extend(found_chapters)

    return list(set(chapter_titles))  # Remove duplicates

# Specify your PDF path here
pdf_path = '/Users/shruti/Desktop/Final_Project/books/The-Courage-To-Be-Disliked.pdf'
chapter_titles = extract_chapters_from_index(pdf_path)

for title in chapter_titles:
    print(title)


THE FIRST NIGHT:
People Always Choose Not to Change
Authors’ Note
All Problems Are Interpersonal Relationship Problems
People Fabricate Anger
Trauma Does Not Exist
Unhappiness Is Something You Choose for Yourself
Socrates and Adler
Are You Okay Just As You Are?
Deny Trauma
Introduction
THE SECOND NIGHT:
Why You Dislike Yourself
Why People Can Change
Your Life Is Decided Here and Now
Feelings of Inferiority Are Subjective Assumptions
The Unknown Third Giant
An Inferiority Complex Is an Excuse
How to Live Without Being Controlled by the Past


In [16]:
!pip install EbookLib beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 550.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for EbookLib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=adb7a0b669c039138e60a317170a19fd93d1f023e7c8799a6e578867cef03bbb
  Stored in directory: /Users/shruti/Library/Caches/pip/wheels/6d/c5/f9/c615f68c24f539a89184ea938cd630d5973d582af6ced69140
Successfully built EbookLib


In [30]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

def extract_chapter_titles_from_index(epub_path):
    book = epub.read_epub(epub_path)
    index_headings = ['contents', 'chapters', 'index']  # Common index page headings
    chapter_titles = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.content, 'html.parser')
            # Check if this document is an index page
            if any(heading in soup.text.lower() for heading in index_headings):
                # Extract potential chapter titles
                links = soup.find_all('a')
                for link in links:
                    chapter_title = link.get_text().strip()
                    if chapter_title:
                        chapter_titles.append(chapter_title)
                break  # Assuming only one index page

    return chapter_titles

# Specify your EPUB path here
epub_path = '/Users/shruti/Desktop/Final_Project/books/carroll-alice-in-wonderland-illustrations.epub'
chapter_titles = extract_chapter_titles_from_index(epub_path)

for title in chapter_titles:
    print(title)


I Down the Rabbit-hole
II The Pool of Tears
III A Caucus-race and a Long Tale
IV The Rabbit Sends in a Little Bill
V Advice From a Caterpillar
VI Pig and Pepper
VII A Mad Tea-party
VIII The Queen's Croquet Ground
IX Who Stole the Tarts?
X Alice's Evidence


In [32]:
def get_chapter_content(epub_path, chapter_title):
    book = epub.read_epub(epub_path)
    chapters_content = {}
    
    # Iterate through each item in the EPUB file
    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(item.get_content(), 'html.parser')
        # Find all the header tags that might indicate a chapter start
        for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
            title = header.get_text(strip=True)
            if title:
                chapters_content[title] = soup.get_text()

    # Now chapters_content has all chapters' text indexed by their title
    return chapters_content.get(chapter_title, "Chapter not found.")

# Example usage
epub_path = '/Users/shruti/Desktop/Final_Project/books/carroll-alice-in-wonderland-illustrations.epub'
chapter_title = 'V Advice From a Caterpillar'  # The title of the chapter the user selected from the dropdown

# Extract the text of the chapter
chapter_text = get_chapter_content(epub_path, chapter_title)
print(chapter_text)











V Advice From a Caterpillar



At last the Caterpillar took the hookah out of its mouth and addressed
Alice in a languid, sleepy voice.
"Who are you?" said the Caterpillar.



Alice replied, rather shyly, "I—I hardly know, sir, just at present—at
least I know who I was when I got up this morning, but I think I must
have changed several times since then."
"What do you mean by that?" said the Caterpillar, sternly. "Explain
yourself!"
"I can't explain myself, I'm afraid, sir," said Alice, "because I'm
not myself, you see—being so many different sizes in a day is very
confusing." She drew herself up and said very gravely, "I think you
ought to tell me who you are, first."
"Why?" said the Caterpillar.
As Alice could not think of any good reason and the Caterpillar seemed
to be in a very unpleasant state of mind, she turned away.
"Come back!" the Caterpillar called after her. "I've something important
to say!" Alice turned and came back again.
"Keep your temper," said the Caterpill

In [35]:
sentences = sent_tokenize(chapter_text)

# Determining the length of each part
total_sentences = len(sentences)
part_length = math.ceil(total_sentences / 4)

# Dividing the sentences into four parts
parts = [sentences[i:i + part_length] for i in range(0, total_sentences, part_length)]

# Print each part
for i, part in enumerate(parts):
    print(f"Part {i+1}: {' '.join(part)}\n")

Part 1: 









V Advice From a Caterpillar



At last the Caterpillar took the hookah out of its mouth and addressed
Alice in a languid, sleepy voice. "Who are you?" said the Caterpillar. Alice replied, rather shyly, "I—I hardly know, sir, just at present—at
least I know who I was when I got up this morning, but I think I must
have changed several times since then." "What do you mean by that?" said the Caterpillar, sternly. "Explain
yourself!" "I can't explain myself, I'm afraid, sir," said Alice, "because I'm
not myself, you see—being so many different sizes in a day is very
confusing." She drew herself up and said very gravely, "I think you
ought to tell me who you are, first." "Why?" said the Caterpillar. As Alice could not think of any good reason and the Caterpillar seemed
to be in a very unpleasant state of mind, she turned away. "Come back!" the Caterpillar called after her. "I've something important
to say!" Alice turned and came back again. "Keep your temper," said the Cate

In [36]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [44]:
from openai import OpenAI
client = OpenAI()

def get_questions(text):

        response = client.chat.completions.create(
        model="gpt-4",
        messages=[
        {
        "role": "system",
        "content": """this text is a story, how to chunk this text like this when you want to use those chunks to form prompts for image generation to form a comic strip out of the whole text. Give me image prompts in such a way that I can create a sensible comic strip out of those image prompts. Give more number of chunks in format: \n 1. Image prompt: "Text example"
2. Image prompt: "Text example"
3. Image prompt: "Text example"\n"""
        },
        {
        "role": "user",
        "content": f"\"\"\"\n{text}\n\"\"\""
        }
        ],
        temperature=1,
        max_tokens=800,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

        first_choice = response.choices[0]
        message_content = first_choice.message.content

        print("==========")
        print(message_content)

        return message_content


ImportError: cannot import name 'OpenAI' from 'openai' (/Users/shruti/opt/anaconda3/envs/venv/lib/python3.8/site-packages/openai/__init__.py)